In [33]:
import os, sys, re, datetime, shutil, copy, subprocess
import numpy as np
import astropy.units as u
from astropy.table import Table
from collections import OrderedDict
sys.path.insert(1, os.path.join(os.path.dirname(os.getcwd()), 'utils'))
radex_calc_path = os.path.join(os.path.dirname(os.getcwd()), 'tmp', 'calc_radex')

/Users/dzliu/Work/2020_Liu_CO_CI/Excitation_Analysis/CICOExcitation/scripts
/Users/dzliu/Work/2020_Liu_CO_CI/Excitation_Analysis/CICOExcitation/scripts


In [10]:
import sqlite3 as sqlite
db_file = os.path.join(radex_calc_path, 'calc_radex.db')
db_conn = sqlite.connect(db_file)
db_cursor = db_conn.cursor()

In [28]:
sqlcmd = "SELECT * FROM CI WHERE T_kin=25 AND d_V=10 AND n_H2=1e3 AND J_UP=1 AND J_LOW=0 " + \
         "ORDER BY ROWID ASC LIMIT 100"
db_cursor.execute(sqlcmd)
queried_rows = db_cursor.fetchall()
print(queried_rows[0])

('calc_T_CMB_2.73_T_kin_25.0_N_CI_5.00e15_n_H2_1.0e3_d_V_10.0_Ju_1_Jl_0', 'calc_T_CMB_2.73_T_kin_25.0_N_CI_5.00e15_n_H2_1.0e3_d_V_10.0', 'CI', 2.72548, 25.0, 5000000000000000.0, 1000.0, 10.0, 1.0, 0.0, 23.6, 492.1607, 609.1354, 16.55, 0.004098, 0.03049, 0.3888, 0.54, 0.3245, 4.982e-07)


In [29]:
columns = [desc[0] for desc in db_cursor.description]

calc_out = OrderedDict([ 
    ('J_UP',[]), 
    ('J_LOW',[]), 
    ('E_UP',[]), 
    ('FREQ',[]), 
    ('WAVE',[]), 
    ('T_ex',[]), 
    ('tau_0',[]), 
    ('T_RJ',[]), 
    ('Pop_u',[]), 
    ('Pop_l',[]), 
    ('Flux_Kkms',[]), 
    ('Flux_ergcm2s',[]), 
]) # must have the same columns and order as in self.create_table

queried_dict = OrderedDict()
for i, col in enumerate(columns):
    queried_dict[col] = [row[i] for row in queried_rows]

queried_table = Table(queried_dict)
queried_table['N_X'].format = '%.3e'
queried_table['n_H2'].format = '%.1e'
queried_table['FREQ'].format = '%.4f'
queried_table['WAVE'].format = '%.4f'
queried_table['T_ex'].format = '%.2f'
queried_table['tau_0'].format = '%.4f'
queried_table['T_RJ'].format = '%.2f'
queried_table['Pop_u'].format = '%.2f'
queried_table['Pop_l'].format = '%.2f'
queried_table

calc_name_Ju_Jl,calc_name,species,T_CMB,T_kin,N_X,n_H2,d_V,J_UP,J_LOW,E_UP,FREQ,WAVE,T_ex,tau_0,T_RJ,Pop_u,Pop_l,Flux_Kkms,Flux_ergcm2s
str69,str59,str2,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
calc_T_CMB_2.73_T_kin_25.0_N_CI_5.00e15_n_H2_1.0e3_d_V_10.0_Ju_1_Jl_0,calc_T_CMB_2.73_T_kin_25.0_N_CI_5.00e15_n_H2_1.0e3_d_V_10.0,CI,2.72548,25.0,5.000e+15,1.0e+03,10.0,1.0,0.0,23.6,492.1607,609.1354,16.55,0.0041,0.03,0.39,0.54,0.3245,4.982e-07
calc_T_CMB_2.73_T_kin_25.0_N_CI_8.89e15_n_H2_1.0e3_d_V_10.0_Ju_1_Jl_0,calc_T_CMB_2.73_T_kin_25.0_N_CI_8.89e15_n_H2_1.0e3_d_V_10.0,CI,2.72548,25.0,8.891e+15,1.0e+03,10.0,1.0,0.0,23.6,492.1607,609.1354,16.55,0.0073,0.05,0.39,0.54,0.5762,8.846e-07
calc_T_CMB_2.73_T_kin_25.0_N_CI_1.58e16_n_H2_1.0e3_d_V_10.0_Ju_1_Jl_0,calc_T_CMB_2.73_T_kin_25.0_N_CI_1.58e16_n_H2_1.0e3_d_V_10.0,CI,2.72548,25.0,1.581e+16,1.0e+03,10.0,1.0,0.0,23.6,492.1607,609.1354,16.55,0.0130,0.10,0.39,0.54,1.022,1.569e-06
calc_T_CMB_2.73_T_kin_25.0_N_CI_2.81e16_n_H2_1.0e3_d_V_10.0_Ju_1_Jl_0,calc_T_CMB_2.73_T_kin_25.0_N_CI_2.81e16_n_H2_1.0e3_d_V_10.0,CI,2.72548,25.0,2.812e+16,1.0e+03,10.0,1.0,0.0,23.6,492.1607,609.1354,16.62,0.0229,0.17,0.39,0.54,1.814,2.785e-06
calc_T_CMB_2.73_T_kin_25.0_N_CI_5.00e16_n_H2_1.0e3_d_V_10.0_Ju_1_Jl_0,calc_T_CMB_2.73_T_kin_25.0_N_CI_5.00e16_n_H2_1.0e3_d_V_10.0,CI,2.72548,25.0,5.000e+16,1.0e+03,10.0,1.0,0.0,23.6,492.1607,609.1354,16.68,0.0406,0.30,0.39,0.54,3.207,4.923e-06
calc_T_CMB_2.73_T_kin_25.0_N_CI_8.89e16_n_H2_1.0e3_d_V_10.0_Ju_1_Jl_0,calc_T_CMB_2.73_T_kin_25.0_N_CI_8.89e16_n_H2_1.0e3_d_V_10.0,CI,2.72548,25.0,8.891e+16,1.0e+03,10.0,1.0,0.0,23.6,492.1607,609.1354,16.77,0.0718,0.53,0.39,0.54,5.636,8.652e-06
calc_T_CMB_2.73_T_kin_25.0_N_CI_1.58e17_n_H2_1.0e3_d_V_10.0_Ju_1_Jl_0,calc_T_CMB_2.73_T_kin_25.0_N_CI_1.58e17_n_H2_1.0e3_d_V_10.0,CI,2.72548,25.0,1.581e+17,1.0e+03,10.0,1.0,0.0,23.6,492.1607,609.1354,16.93,0.1264,0.92,0.40,0.53,9.827,1.509e-05
calc_T_CMB_2.73_T_kin_25.0_N_CI_2.81e17_n_H2_1.0e3_d_V_10.0_Ju_1_Jl_0,calc_T_CMB_2.73_T_kin_25.0_N_CI_2.81e17_n_H2_1.0e3_d_V_10.0,CI,2.72548,25.0,2.812e+17,1.0e+03,10.0,1.0,0.0,23.6,492.1607,609.1354,17.21,0.2206,1.59,0.40,0.53,16.89,2.593e-05
calc_T_CMB_2.73_T_kin_25.0_N_CI_5.00e17_n_H2_1.0e3_d_V_10.0_Ju_1_Jl_0,calc_T_CMB_2.73_T_kin_25.0_N_CI_5.00e17_n_H2_1.0e3_d_V_10.0,CI,2.72548,25.0,5.000e+17,1.0e+03,10.0,1.0,0.0,23.6,492.1607,609.1354,17.66,0.3809,2.66,0.41,0.52,28.34,4.351e-05


In [6]:
db_conn.close()